# Identify relevant law articles for each case

In [60]:
import yaml

with open("../../src/config/cfg.yaml", 'r') as stream:
    config = yaml.safe_load(stream)

## Validate schema of received json

In [61]:
import json

f = open('../../data/01_raw/dev/sachverhalt.json')
data = json.load(f)

print(data)

{'beteiligter': 'Sample Name', 'fahrzeug': 'Sample Vehicle', 'aktionen': [{'id': 1, 'beschreibung': 'Sample Description'}, {'id': 2, 'beschreibung': 'Sample Description 2'}]}


In [62]:
import sys
# add path to sys.path in order to access schemas
sys.path.append('../../src/schemas')

from sachverhalt import schema_sachverhalt

schema_sachverhalt.validate(data)

## Check case
for each aktion in sachverhalt
* vectorize aktion
* search best matching law article for aktion
* add id to aktion (attr: artikel_id)

### setup pinecone connection

In [63]:
import os
import pinecone

api_key = os.environ["PINECONE_API_KEY"]
pinecone.init(api_key=api_key, environment=config['vectorization']['environment'])

index_name = "law"
index = pinecone.Index(index_name)

### load vectorstore

In [64]:
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.embeddings.huggingface import HuggingFaceEmbeddings

openai_api_key = os.environ.get('OPENAI_API_KEY')
model_name = config['sentence_transformer']['model_name']

embed = HuggingFaceEmbeddings(
    model_name=model_name
)
print(embed)

client=SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False})
  (2): Normalize()
) model_name='all-MiniLM-L6-v2' cache_folder=None model_kwargs={} encode_kwargs={}


In [65]:
from langchain.vectorstores import Pinecone

text_field = "text"

# switch back to normal index for langchain
index = pinecone.Index(index_name)

vectorstore = Pinecone(
    index, embed.embed_query, text_key=text_field
)

In [66]:
import sys
sys.path.append('../../src/features')

from preprocess import remove_stopwords, lemmatize

In [67]:
query = "Missachtung des Vortritts und Kollision mit rechter Fahrzeugseite von G"
query = remove_stopwords(query)
query = lemmatize(query)

print(query)

vectorstore.similarity_search(
    query,  # our search query
    k=10  # return 10 most relevant docs
)

Missachtung Vortritt Kollision rechter Fahrzeugseite G


[Document(page_content='geregelt Fall Beispiel Verzweigung zugleich Richtung Fahrzeug eintreffen Führer besonders vorsichtig fahren Vortritt verständigen', metadata={}),
 Document(page_content='ausserorts abgestellt mehrspurig Fahrzeug Standlicht Parklicht Seite Verkehr verwenden Fahrzeug derartig Licht entsprechend Fahrzeugart vorgeschrieben Licht verwenden', metadata={}),
 Document(page_content='Benützer fahrzeugähnlich Gerät gelten Fussgänger anwendbar Verkehrsregel', metadata={}),
 Document(page_content='platzsparend parkieren dürfen Wegfahrt Fahrzeug behindern', metadata={}),
 Document(page_content='Zeichengebung entbinden Fahrzeugführer geboten Vorsicht', metadata={}),
 Document(page_content='Längsstreifen Fussgänger 6.19 dürfen Fahrzeug benützen Fussgängerverkehr behindern', metadata={}),
 Document(page_content='steil Strasse Bergstrasse gleichartig Fahrzeug * kreuzen abwärtsfahrend zurückfahren ausser befinden nahe Ausweichstelle Kreuzen ungleichartig Fahrzeug gelten Artikel 9 